In [ ]:
import pandas as pd
from datetime import timedelta
import ast
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
movies_df = pd.read_csv('imdb_movies.csv')

In [ ]:
movies_df.head()

In [ ]:
movies_df.info()

In [ ]:
movies_df.describe(include='all')

In [ ]:
movies_with_no_duration = movies_df[movies_df['duration'].isnull()]
movies_with_no_duration.to_csv('movies_with_no_duration.csv', index=False)
movies_df = movies_df.dropna(subset=['duration'])

In [ ]:
movies_with_no_rating = movies_df[movies_df['imdb_rating'].isnull()]
movies_with_no_rating.to_csv('movies_with_no_rating.csv', index=False)
movies_df = movies_df.dropna(subset=['imdb_rating'])

In [ ]:
movies_with_more_than_8_rating = movies_df[movies_df['imdb_rating'] > 8]
movies_with_more_than_8_rating.to_csv('movies_with_more_than_8_rating.csv', index=False)
movies_df = movies_df[movies_df['imdb_rating'] <= 8]

In [ ]:
movies_df['release_date'] = movies_df['release_date'].apply(lambda x: x.split('(')[0].strip().strip())
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], format='mixed')

In [ ]:
movies_df['duration'] = movies_df['duration'].replace('PT', '', regex=True)

In [ ]:
def str_to_time(duration):
    hours = int(duration.split('H')[0]) if 'H' in duration else 0
    minutes = int(duration.split('M')[0].split('H')[-1]) if 'M' in duration else 0
    return f"{hours:02}:{minutes:02}:00"

In [ ]:
movies_df['duration'] = movies_df['duration'].apply(str_to_time)
movies_df['duration'] = pd.to_datetime(movies_df['duration'], format='%H:%M:%S').dt.time

In [ ]:
actors = list(pd.read_csv('Actors.csv')['Actors'])
actresses = list(pd.read_csv('Actresses.csv')['Actresses'])

In [ ]:
movies_df['cast'] = movies_df['cast'].apply(ast.literal_eval)
movies_df["actor"] = movies_df["cast"].apply(lambda x: [name for name in x if name in actors])
movies_df["actress"] = movies_df["cast"].apply(lambda x: [name for name in x if name in actresses])

In [ ]:
movies_df.drop(columns=['cast'], inplace=True)

In [ ]:
movies_df['directors'] = movies_df['directors'].apply(lambda x: ast.literal_eval(x))
movies_df['writers'] = movies_df['writers'].apply(lambda x: ast.literal_eval(x))
movies_df['genres'] = movies_df['genres'].apply(lambda x: ast.literal_eval(x))
movies_df['languages'] = movies_df['languages'].apply(lambda x: ast.literal_eval(x))
movies_df['production_companies'] = movies_df['production_companies'].apply(lambda x: ast.literal_eval(x))

In [ ]:
movies_df.info()

In [ ]:
movies_df.describe(include='all')

In [ ]:
movies_df.to_csv('cleaned_imdb_movies.csv', index=False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
sns.histplot(movies_df['imdb_rating'], kde=True, ax=ax[0])
sns.boxplot(movies_df['imdb_rating'], ax=ax[1], orient='h')
plt.show()

In [ ]:
if 'duration_minutes' not in movies_df.columns:
	movies_df['duration_minutes'] = movies_df['duration'].apply(lambda x: x.hour * 60 + x.minute)

plt.figure(figsize=(20, 10))
sns.histplot(movies_df['duration_minutes'], kde=True)
plt.show()

In [ ]:
top_10_imdb_rated_movies = movies_df.nlargest(20, 'imdb_rating')

plt.figure(figsize=(12, 8))
sns.barplot(x='imdb_rating', y='title', data=top_10_imdb_rated_movies)
plt.xlabel('IMDb Rating')
plt.ylabel('Movies')
plt.title('Top 10 IMDb Rated Movies')
plt.show()

In [ ]:
movies_df['duration_minutes'] = movies_df['duration'].apply(lambda x: x.hour * 60 + x.minute)

# Plot the bar chart
plt.figure(figsize=(20, 50))
sns.barplot(x='duration_minutes', y='title', data=movies_df, order=movies_df.sort_values('duration_minutes', ascending=False)['title'], orient='h')
plt.xticks(rotation=90)
plt.xlabel('Duration (minutes)')
plt.ylabel('Title')
plt.title('Duration of Movies')
plt.show()

In [ ]:
def plot_top_ten(column, column_name):
    plt.figure(figsize=(10,8))
    sns.barplot(x=column.values, hue=column.index, y=column.index)
    plt.xlabel('Movie count')
    plt.ylabel(column_name)
    plt.title(f'Top 10 {column_name}')
    plt.show()

In [ ]:
top_10_directors = movies_df['directors'].explode().value_counts().head(10)
plot_top_ten(top_10_directors, 'Directors')

In [ ]:
top_10_writers = movies_df['writers'].explode().value_counts().head(10)
plot_top_ten(top_10_writers, "Writers")

In [ ]:
top_10_genres = movies_df['genres'].explode().value_counts().head(10)
plot_top_ten(top_10_genres, "Genres")

In [ ]:
top_10_languages = movies_df['languages'].explode().value_counts().head(10)
plot_top_ten(top_10_languages, "Languages")

In [ ]:
top_10_actors = movies_df['actor'].explode().value_counts().head(10)
plot_top_ten(top_10_actors, "Actors (Including Main, Supporting, Cameo)")

In [ ]:
top_10_actresses = movies_df['actress'].explode().value_counts().head(10)
plot_top_ten(top_10_actresses, "Actresses (Including Main, Supporting, Cameo)")

In [ ]:
plt.figure(figsize=(20, 10))
sns.boxplot(x='imdb_rating', y='genres', data=movies_df.explode('genres'))
plt.xlabel('IMDB Rating')
plt.ylabel('Genres')
plt.title('Genres vs IMDB Rating')
plt.show()

In [ ]:
movies_directors_df = movies_df[['title', 'directors']].explode('directors')
movies_directors_df.to_excel('movies_directors.xlsx', index=False)

movies_writers_df = movies_df[['title', 'writers']].explode('writers')
movies_writers_df.to_excel('movies_writers.xlsx', index=False)

movies_genres_df = movies_df[['title', 'genres']].explode('genres')
movies_genres_df.to_excel('movies_genres.xlsx', index=False)

movies_languages_df = movies_df[['title', 'languages']].explode('languages')
movies_languages_df.to_excel('movies_languages.xlsx', index=False)

movies_production_companies_df = movies_df[['title', 'production_companies']].explode('production_companies')
movies_production_companies_df.to_excel('movies_production_companies.xlsx', index=False)

movies_actors_df = movies_df[['title', 'actor']].explode('actor')
movies_actors_df.to_excel('movies_actors.xlsx', index=False)

movies_actresses_df = movies_df[['title', 'actress']].explode('actress')
movies_actresses_df.to_excel('movies_actresses.xlsx', index=False)

In [ ]:
movies_info_df = movies_df[['title', 'release_date', 'duration', 'imdb_rating', 'duration_minutes']]
movies_info_df.to_excel('movies_info.xlsx', index=False)

In [ ]:
no_directors_df = movies_df[movies_df['directors'].apply(lambda x: len(x) == 0)]
no_directors_df

In [ ]:
no_writers_df = movies_df[movies_df['writers'].apply(lambda x: len(x) == 0)]
no_writers_df

In [ ]:
movies_df = movies_df[movies_df['directors'].apply(lambda x: len(x) > 0)]

In [ ]:
movies_df = movies_df.explode('directors').explode('writers').explode('genres').explode('languages').explode('production_companies').explode('actor').explode('actress')

In [ ]:
movies_df.info()

In [ ]:
movies_df.describe(include='all')

In [ ]:
movies_df.to_csv('exploded_imdb_movies.csv', index=False)